In [4]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.estimator import Estimator
from sagemaker.model import Model
from sagemaker.inputs import TrainingInput
from sagemaker.model_monitor import DataCaptureConfig

from sagemaker import get_execution_role

# role = "<sagemaker-execution-role>"

role=get_execution_role()
bucket = "feature-engineering-bucket-989220949c9c"
region = "ap-south-1"
image_uri_train = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-train:latest"

# Preprocessing Step
preprocess_processor = ScriptProcessor(
    image_uri=image_uri_train,
    role=role,
    command=["python3"],
    instance_count=1,
    instance_type="ml.t3.medium"
)

preprocess_step = ProcessingStep(
    name="preprocess_step",
    processor=preprocess_processor,
    code="preprocess.py",
    inputs=[
        ProcessingInput(
            source="s3://feature-engineering-bucket-989220949c9c/Dataset/bank-additional-full.csv",  # S3 input file
            destination="/opt/ml/processing/input/"  # Path inside the container
        )
        # {"source": "s3://feature-engineering-bucket/bank-additional-full.csv", "destination": "/opt/ml/processing/input"}
    ],
    outputs=[
        ProcessingOutput(
            output_name="processed_data",
            source="/opt/ml/processing/output",  # Path inside the container
            destination=f"s3://{bucket}/processed/"  # S3 output location
        )
    ]
)


# Define Estimator
estimator = Estimator(
    image_uri=image_uri_train,
    role=role,
    entry_point="train.py",             # ✅ Training script passed at runtime
    source_dir=".",                     # ✅ Folder where train.py is located
    instance_count=1,
    instance_type="ml.m5.large",
    base_job_name="mlops-train",
    output_path=f"s3://{bucket}/models/"  # ✅ where model.tar.gz will be saved
)

# Define TrainingStep in pipeline
training_step = TrainingStep(
    name="TrainingStep",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=f"s3://{bucket}/processed/bank-additional-processed.csv",
            content_type="text/csv"
        )
    }
)


# Register model
model_artifact = training_step.properties.ModelArtifacts.S3ModelArtifacts
model = Model(
    image_uri=image_uri_train,
    model_data=model_artifact,
    role=role
)

register_step = RegisterModel(
    name="RegisterModel",
    model=model,
    content_types=["text/csv"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name="mlops-model-group",
    approval_status="Approved"
)

# Define Pipeline
pipeline = Pipeline(
    name="MLOpsPipeline",
    steps=[preprocess_step, training_step, register_step]
)

# Deploy Pipeline
pipeline.upsert(role_arn=role)
execution = pipeline.start()
execution.wait()


# --------------------------------------------------------------------------------
# 5. Deploy the Model to Endpoint
# --------------------------------------------------------------------------------

image_uri_inference = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-inference:latest"

import boto3

steps = execution.list_steps()
sm = boto3.client("sagemaker")

training_job_arn = None

for step in steps:
    if step["StepName"] == "TrainingStep":
        training_job_arn = step["Metadata"]["TrainingJob"]["Arn"]
        break

# Extract training job name from ARN
training_job_name = training_job_arn.split("/")[-1]
print("Training job:", training_job_name)
s3_model_path = f"s3://{bucket}/models/{training_job_name}/output/model.tar.gz"
print("s3_model_path:", s3_model_path)

        

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

model = Model(
    image_uri=image_uri_inference,
    model_data=s3_model_path,
    role=role,
    # entry_point="serve.py",       # ✅ Add this line
    # source_dir=".",               # ✅ Or directory containing serve.py
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint15",
    data_capture_config=DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=f"s3://{bucket}/monitoring/data/",
        capture_options=["Input", "Output"]
    )
)

if predictor is None:
    raise RuntimeError("🚨 model.deploy() returned None — deployment likely failed.")

response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print(response)



# --------------------------------------------------------------------------------
# 6. Monitor Performance Using Model Monitor
# --------------------------------------------------------------------------------


from sagemaker.model_monitor import DefaultModelMonitor


monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800
)


monitor.create_monitoring_schedule(
    monitor_schedule_name="mlops-monitor",
    endpoint_input=predictor.endpoint_name,
    output_s3_uri=f"s3://{bucket}/monitoring/reports/",
    # generate the report every 5 minutes
    schedule_cron_expression="cron(0/5 * * * ? *)"
)

# Now performance reports will be generated daily in S3


# --------------------------------------------------------------------------------
# 7. Automate Retraining (Optional)
# --------------------------------------------------------------------------------


# Use EventBridge or S3 trigger → Lambda → start pipeline
# Lambda code (Python) can call:
# import boto3
# sm = boto3.client('sagemaker')
# sm.start_pipeline_execution(PipelineName='MLOpsPipeline')


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.SecurityGroupIds
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.TrainingJob.VpcConfig.Subn

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:155                                                                                  │
│                                                                                                  │
│   152 )                                                                                          │
│   153                                                                                            │
│   154 if predictor is None:                                                                      │
│ ❱ 155 │   raise RuntimeError("🚨 model.deploy() returned None — deployment likely failed.")      │
│   156                                                                                            │
│   157 response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,2   │
│   158 print(response)                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: 🚨 model.deploy() returned None — deployment likely failed.

In [6]:
if predictor is None:
    raise RuntimeError("🚨 model.deploy() returned None — deployment likely failed.")

response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print(response)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 if predictor is None:                                                                        │
│ ❱ 2 │   raise RuntimeError("🚨 model.deploy() returned None — deployment likely failed.")        │
│   3                                                                                              │
│   4 response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,2     │
│   5 print(response)                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: 🚨 model.deploy() returned None — deployment likely failed.

In [ ]:

image_uri_inference = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-inference:latest"

import boto3

steps = execution.list_steps()
sm = boto3.client("sagemaker")

training_job_arn = None

for step in steps:
    if step["StepName"] == "TrainingStep":
        training_job_arn = step["Metadata"]["TrainingJob"]["Arn"]
        break

# Extract training job name from ARN
training_job_name = training_job_arn.split("/")[-1]
print("Training job:", training_job_name)
s3_model_path = f"s3://{bucket}/models/{training_job_name}/output/model.tar.gz"
print("s3_model_path:", s3_model_path)

        

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

model = Model(
    image_uri=image_uri_inference,
    model_data=s3_model_path,
    role=role,
    entry_point="serve.py",       # ✅ Add this line
    source_dir=".",               # ✅ Or directory containing serve.py
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint16",
    data_capture_config=DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=f"s3://{bucket}/monitoring/data/",
        capture_options=["Input", "Output"]
    )
)

if predictor is None:
    raise RuntimeError("🚨 model.deploy() returned None — deployment likely failed.")

response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print(response)



In [ ]:
import boto3


sm = boto3.client("sagemaker")

# Step 1: Get the latest approved model package
latest_model_package = sm.list_model_packages(
    ModelPackageGroupName="mlops-model-group",
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1
)["ModelPackageSummaryList"][0]["ModelPackageArn"]

from sagemaker import ModelPackage
mp = ModelPackage(
    role=role,
    model_package_arn=latest_model_package
)

predictor = mp.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint19"
)

if predictor is None:
    raise RuntimeError("🚨 model.deploy() returned None — deployment likely failed.")

response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print(response)



<h1>Pipeline where the inference uses the registry model instead of the trained model</h1> 

In [ ]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.estimator import Estimator
from sagemaker.model import Model
from sagemaker.inputs import TrainingInput
from sagemaker.model_monitor import DataCaptureConfig

from sagemaker import get_execution_role

# role = "<sagemaker-execution-role>"

role=get_execution_role()
bucket = "feature-engineering-bucket-989220949c9c"
region = "ap-south-1"
image_uri_train = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-train:latest"
image_uri_inference = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-inference:latest"

# Preprocessing Step
preprocess_processor = ScriptProcessor(
    image_uri=image_uri_train,
    role=role,
    command=["python3"],
    instance_count=1,
    instance_type="ml.t3.medium"
)

preprocess_step = ProcessingStep(
    name="preprocess_step",
    processor=preprocess_processor,
    code="preprocess.py",
    inputs=[
        ProcessingInput(
            source="s3://feature-engineering-bucket-989220949c9c/Dataset/bank-additional-full.csv",  # S3 input file
            destination="/opt/ml/processing/input/"  # Path inside the container
        )
        # {"source": "s3://feature-engineering-bucket/bank-additional-full.csv", "destination": "/opt/ml/processing/input"}
    ],
    outputs=[
        ProcessingOutput(
            output_name="processed_data",
            source="/opt/ml/processing/output",  # Path inside the container
            destination=f"s3://{bucket}/processed/"  # S3 output location
        )
    ]
)


# Define Estimator
estimator = Estimator(
    image_uri=image_uri_train,
    role=role,
    entry_point="train.py",             # ✅ Training script passed at runtime
    # source_dir=".",                     # ✅ Folder where train.py is located
    instance_count=1,
    instance_type="ml.m5.large",
    base_job_name="mlops-train",
    output_path=f"s3://{bucket}/models/"  # ✅ where model.tar.gz will be saved
)

# Define TrainingStep in pipeline
training_step = TrainingStep(
    name="TrainingStep",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=f"s3://{bucket}/processed/bank-additional-processed.csv",
            content_type="text/csv"
        )
    }
)


# Register model
model_artifact = training_step.properties.ModelArtifacts.S3ModelArtifacts
model = Model(
    image_uri=image_uri_inference,
    model_data=model_artifact,
    role=role
)

register_step = RegisterModel(
    name="RegisterModel",
    model=model,
    content_types=["text/csv"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name="mlops-model-group",
    approval_status="Approved"
)

# Define Pipeline
pipeline = Pipeline(
    name="MLOpsPipeline",
    steps=[preprocess_step, training_step, register_step]
)

# Deploy Pipeline
pipeline.upsert(role_arn=role)
execution = pipeline.start()
execution.wait()


# --------------------------------------------------------------------------------
# 5. Deploy the Model to Endpoint
# --------------------------------------------------------------------------------

import boto3
import sagemaker
sagemaker_session = sagemaker.Session()

steps = execution.list_steps()
sm = boto3.client("sagemaker")

# Step 1: Get the latest approved model package
latest_model_package = sm.list_model_packages(
    ModelPackageGroupName="mlops-model-group",
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1
)["ModelPackageSummaryList"][0]["ModelPackageArn"]

from sagemaker import ModelPackage
mp = ModelPackage(
    role=role,
    model_package_arn=latest_model_package
)


from sagemaker.model_monitor import DataCaptureConfig

data_capture = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=f"s3://{bucket}/monitoring/data/",
    capture_options=["Input", "Output"]
)



predictor = mp.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint-33",
    data_capture_config=data_capture
)

# If SDK returns None, manually create predictor
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.deserializers import StringDeserializer

if predictor is None:
    # predictor = Predictor(endpoint_name="mlops-prod-endpoint-20", sagemaker_session=sagemaker_session)

    predictor = Predictor(
        endpoint_name="mlops-prod-endpoint-33",
        sagemaker_session=sagemaker_session,
        serializer=CSVSerializer(),
        deserializer=JSONDeserializer(),
        # deserializer=StringDeserializer(),
    )


response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print("prediction results by the endpoint:\n")
print(response)
print("Using deserializer:", type(predictor.deserializer))

# --------------------------------------------------------------------------------
# 6. Monitor Performance Using Model Monitor
# --------------------------------------------------------------------------------


from sagemaker.model_monitor import DefaultModelMonitor


monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.t3.medium",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800
)


monitor.create_monitoring_schedule(
    monitor_schedule_name="mlops-monitor",
    endpoint_input=predictor.endpoint_name,
    output_s3_uri=f"s3://{bucket}/monitoring/reports/",
    # sagemaker does not support every 5 minutes, the minimum unit is hour
    schedule_cron_expression="cron(0 * * * ? *)"  # ✔ every hour
)

# Now performance reports will be generated daily in S3


# --------------------------------------------------------------------------------
# 7. Automate Retraining (Optional)
# --------------------------------------------------------------------------------


# Use EventBridge or S3 trigger → Lambda → start pipeline
# Lambda code (Python) can call:
# import boto3
# sm = boto3.client('sagemaker')
# sm.start_pipeline_execution(PipelineName='MLOpsPipeline')


<h1>Monitoring part</h1>

<h1>Deployment part</h1>

In [28]:

import boto3
import sagemaker
sagemaker_session = sagemaker.Session()

steps = execution.list_steps()
sm = boto3.client("sagemaker")

# Step 1: Get the latest approved model package
latest_model_package = sm.list_model_packages(
    ModelPackageGroupName="mlops-model-group",
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1
)["ModelPackageSummaryList"][0]["ModelPackageArn"]

from sagemaker import ModelPackage
mp = ModelPackage(
    role=role,
    model_package_arn=latest_model_package
)


from sagemaker.model_monitor import DataCaptureConfig

data_capture = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=f"s3://{bucket}/monitoring/data/",
    capture_options=["Input", "Output"]
)



predictor = mp.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint-33",
    data_capture_config=data_capture
)

# If SDK returns None, manually create predictor
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.deserializers import StringDeserializer

if predictor is None:
    # predictor = Predictor(endpoint_name="mlops-prod-endpoint-20", sagemaker_session=sagemaker_session)

    predictor = Predictor(
        endpoint_name="mlops-prod-endpoint-33",
        sagemaker_session=sagemaker_session,
        serializer=CSVSerializer(),
        deserializer=JSONDeserializer(),
        # deserializer=StringDeserializer(),
    )


response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print("prediction results by the endpoint:\n")
print(response)
print("Using deserializer:", type(predictor.deserializer))

# --------------------------------------------------------------------------------
# 6. Monitor Performance Using Model Monitor
# --------------------------------------------------------------------------------


from sagemaker.model_monitor import DefaultModelMonitor


monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.t3.medium",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800
)


monitor.create_monitoring_schedule(
    monitor_schedule_name="mlops-monitor",
    endpoint_input=predictor.endpoint_name,
    output_s3_uri=f"s3://{bucket}/monitoring/reports/",
    # sagemaker does not support every 5 minutes, the minimum unit is hour
    schedule_cron_expression="cron(0 * * * ? *)"  # ✔ every hour
)


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.Model.VpcConfig
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
------!prediction results by the endpoint:

[False]
Using deserializer: <class 'sagemaker.base_deserializers.JSONDeserializer'>
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemak

In [21]:
import boto3
sm = boto3.client("sagemaker")

desc = sm.describe_endpoint(EndpointName="mlops-prod-endpoint-29")
print(desc["EndpointStatus"])
print(desc["ProductionVariants"])

InService
[{'VariantName': 'AllTraffic', 'DeployedImages': [{'SpecifiedImage': '961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-inference@sha256:3b9259701464fea4c82d6bd679d296e05ef99f2fa7907882c00fa41279e613da', 'ResolvedImage': '961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-inference@sha256:3b9259701464fea4c82d6bd679d296e05ef99f2fa7907882c00fa41279e613da', 'ResolutionTime': datetime.datetime(2025, 12, 10, 11, 33, 2, 729000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}]


In [ ]:
execution.list_steps()


<h1>Delete monitoring</h1>
To delete the monitoring, you can do the following operations:
<h2>Option 1 — Delete the Monitoring Schedule (recommended)</h2>

Effect:

✔ Stops monitoring

✔ Does not delete endpoint

✔ Safe and reversible (you can recreate later)


In [ ]:
import boto3
sm = boto3.client("sagemaker")

sm.delete_monitoring_schedule(
    MonitoringScheduleName="mlops-monitor"
)


<h2>Option 2 — Stop / Pause the Monitoring Schedule</h2>
If you want to temporarily disable:

In [ ]:
import boto3
sm = boto3.client("sagemaker")

sm.stop_monitoring_schedule(MonitoringScheduleName="mlops-monitor")


Effect:

✔ Stops future monitoring jobs

✔ Schedule remains (can be restarted)

To resume later:

In [ ]:
sm.start_monitoring_schedule(MonitoringScheduleName="mlops-monitor")


<h2>Option 3 — Delete the Endpoint (not required unless you want to)</h2>
If you want to remove all cost from hosting the model:

In [ ]:
sm.delete_endpoint(EndpointName="mlops-prod-endpoint-30")
sm.delete_endpoint_config(EndpointConfigName="mlops-prod-endpoint-30")


But note:

🚫 This does NOT delete the monitoring schedule.
    
You must still run:

In [ ]:
sm.delete_monitoring_schedule(MonitoringScheduleName="mlops-monitor")


Otherwise AWS will still try to run monitoring jobs → errors.

<h1>Check status of the endpoint via boto3</h1>
If you see:

- Status: Deleting → you must wait until deletion finishes.
- Status: Failed or anything else → force delete or use a new name.

In [ ]:
import boto3
sm = boto3.client('sagemaker')

endpoint_name = 'mlops-prod-endpoint3'

try:
    response = sm.describe_endpoint(EndpointName=endpoint_name)
    print("Status:", response['EndpointStatus'])
except sm.exceptions.ClientError as e:
    if "Could not find" in str(e):
        print("✅ Endpoint does not exist")
    else:
        raise e


<h1>Add the automate retraining</h1>

In [ ]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.estimator import Estimator
from sagemaker.model import Model
from sagemaker.inputs import TrainingInput
from sagemaker.model_monitor import DataCaptureConfig

from sagemaker import get_execution_role

# role = "<sagemaker-execution-role>"

role=get_execution_role()
bucket = "feature-engineering-bucket-989220949c9c"
region = "ap-south-1"
image_uri_train = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-train:latest"
image_uri_inference = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-inference:latest"

# Preprocessing Step
preprocess_processor = ScriptProcessor(
    image_uri=image_uri_train,
    role=role,
    command=["python3"],
    instance_count=1,
    instance_type="ml.t3.medium"
)

preprocess_step = ProcessingStep(
    name="preprocess_step",
    processor=preprocess_processor,
    code="preprocess.py",
    inputs=[
        ProcessingInput(
            source="s3://feature-engineering-bucket-989220949c9c/Dataset/bank-additional-full.csv",  # S3 input file
            destination="/opt/ml/processing/input/"  # Path inside the container
        )
        # {"source": "s3://feature-engineering-bucket/bank-additional-full.csv", "destination": "/opt/ml/processing/input"}
    ],
    outputs=[
        ProcessingOutput(
            output_name="processed_data",
            source="/opt/ml/processing/output",  # Path inside the container
            destination=f"s3://{bucket}/processed/"  # S3 output location
        )
    ]
)


# Define Estimator
estimator = Estimator(
    image_uri=image_uri_train,
    role=role,
    entry_point="train.py",             # ✅ Training script passed at runtime
    # source_dir=".",                     # ✅ Folder where train.py is located
    instance_count=1,
    instance_type="ml.m5.large",
    base_job_name="mlops-train",
    output_path=f"s3://{bucket}/models/"  # ✅ where model.tar.gz will be saved
)

# Define TrainingStep in pipeline
training_step = TrainingStep(
    name="TrainingStep",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=f"s3://{bucket}/processed/bank-additional-processed.csv",
            content_type="text/csv"
        )
    }
)


# Register model
model_artifact = training_step.properties.ModelArtifacts.S3ModelArtifacts
model = Model(
    image_uri=image_uri_inference,
    model_data=model_artifact,
    role=role
)

register_step = RegisterModel(
    name="RegisterModel",
    model=model,
    content_types=["text/csv"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name="mlops-model-group",
    approval_status="Approved"
)

# Define Pipeline
pipeline = Pipeline(
    name="MLOpsPipeline",
    steps=[preprocess_step, training_step, register_step]
)

# Deploy Pipeline
pipeline.upsert(role_arn=role)
execution = pipeline.start()
execution.wait()


# --------------------------------------------------------------------------------
# 5. Deploy the Model to Endpoint
# --------------------------------------------------------------------------------

import boto3
import sagemaker
sagemaker_session = sagemaker.Session()

steps = execution.list_steps()
sm = boto3.client("sagemaker")

# Step 1: Get the latest approved model package
latest_model_package = sm.list_model_packages(
    ModelPackageGroupName="mlops-model-group",
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1
)["ModelPackageSummaryList"][0]["ModelPackageArn"]

from sagemaker import ModelPackage
mp = ModelPackage(
    role=role,
    model_package_arn=latest_model_package
)


from sagemaker.model_monitor import DataCaptureConfig

data_capture = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=f"s3://{bucket}/monitoring/data/",
    capture_options=["Input", "Output"]
)



predictor = mp.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint-33",
    data_capture_config=data_capture
)

# If SDK returns None, manually create predictor
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.deserializers import StringDeserializer

if predictor is None:
    # predictor = Predictor(endpoint_name="mlops-prod-endpoint-20", sagemaker_session=sagemaker_session)

    predictor = Predictor(
        endpoint_name="mlops-prod-endpoint-33",
        sagemaker_session=sagemaker_session,
        serializer=CSVSerializer(),
        deserializer=JSONDeserializer(),
        # deserializer=StringDeserializer(),
    )


response = predictor.predict("56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0")  # example input
print("prediction results by the endpoint:\n")
print(response)
print("Using deserializer:", type(predictor.deserializer))

# --------------------------------------------------------------------------------
# 6. Monitor Performance Using Model Monitor
# --------------------------------------------------------------------------------


from sagemaker.model_monitor import DefaultModelMonitor


monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.t3.medium",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800
)


monitor.create_monitoring_schedule(
    monitor_schedule_name="mlops-monitor",
    endpoint_input=predictor.endpoint_name,
    output_s3_uri=f"s3://{bucket}/monitoring/reports/",
    # sagemaker does not support every 5 minutes, the minimum unit is hour
    schedule_cron_expression="cron(0 * * * ? *)"  # ✔ every hour
)

# Now performance reports will be generated daily in S3


# --------------------------------------------------------------------------------
# 7. Automate Retraining (Optional)
# --------------------------------------------------------------------------------


# Use EventBridge or S3 trigger → Lambda → start pipeline
# Lambda code (Python) can call:
# import boto3
# sm = boto3.client('sagemaker')
# sm.start_pipeline_execution(PipelineName='MLOpsPipeline')
